## Installation

In [1]:
import tensorflow as tf

/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
print (tf.__version__)

1.10.1


## Creating your first graph and running it in a session

In [3]:
# create tensor graph
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

In [4]:
# create running session
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


In [5]:
# 使用with声明
with tf.Session() as sess:
    # 这里自动设置了下面代码执行的上下文，即tf.Session()，而且这个上下文会自动释放
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
print(result)

42


In [6]:
# 全局初始化
init = tf.global_variables_initializer() # prepare an init node, 这里只是获取一个初始化函数，并不会跑代码
with tf.Session() as sess:
    init.run() # 初始化所有变量
    result = f.eval()
print(result)

42


In [7]:
# 全局上下文，仅在Python shell中使用
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close() # 这个全局上下文需要手动关闭

42


## Managing Graphs

In [8]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [9]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
x2.graph is graph

True

In [10]:
x2.graph is tf.get_default_graph()

False

In [11]:
tf.reset_default_graph()

In [12]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3
with tf.Session() as sess:
    # 分开计算时，x会被计算两次
    print(y.eval())
    print(z.eval())

10
15


In [13]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z]) # 这种情况下，x只会被计算一次
    print(y_val)
    print(z_val)

10
15


## Linear Regression with Tensorflow

In [3]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data] # add bias feature x0
housing_target = housing.target.reshape(-1, 1);

# 使用Normal Equation进行计算，因此只需要一条公式：theta = (X.transpose * X).inverse * X.transpose * y
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing_target, dtype=tf.float32, name='y')
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()

In [15]:
theta_value

array([[-3.7185181e+01],
       [ 4.3633747e-01],
       [ 9.3952334e-03],
       [-1.0711310e-01],
       [ 6.4479220e-01],
       [-4.0338000e-06],
       [-3.7813708e-03],
       [-4.2348403e-01],
       [-4.3721911e-01]], dtype=float32)

In [16]:
# calculate mean squared error
y_pred = np.matmul(housing_data_plus_bias, theta_value)
mse = np.mean(np.square(y_pred - housing_target))
print("MSE:", mse)

MSE: 0.5243259057328512


使用TensorFlow计算的好处就是，可以利用GPU进行计算加速（如果支持GPU计算的话）

## Implementing Gradient Descent

In [4]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
scaled_housing_data = std_scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [5]:
%%time
n_epochs = 1000
learning_rate = 0.01
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing_target, dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradient = 2 / m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradient)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch', epoch, "MSE =", mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()
print(best_theta)

Epoch 0 MSE = 13.18095
Epoch 100 MSE = 0.87504846
Epoch 200 MSE = 0.70171505
Epoch 300 MSE = 0.65226984
Epoch 400 MSE = 0.6176784
Epoch 500 MSE = 0.59256905
Epoch 600 MSE = 0.5743067
Epoch 700 MSE = 0.56100714
Epoch 800 MSE = 0.55130893
Epoch 900 MSE = 0.5442258
[[ 2.0685523 ]
 [ 0.82384074]
 [ 0.16078205]
 [-0.17397061]
 [ 0.1945243 ]
 [ 0.01086216]
 [-0.04275855]
 [-0.5989245 ]
 [-0.5650062 ]]
CPU times: user 469 ms, sys: 61.9 ms, total: 531 ms
Wall time: 443 ms


In [25]:
# calculate mean squared error
y_pred = np.matmul(scaled_housing_data_plus_bias, best_theta)
mse_value = np.mean(np.square(y_pred - housing_target))
print("MSE:", mse_value)

MSE: 0.5455102101089745


### Using autodiff

In [33]:
%%time
theta2 = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name='theta2')
gradient = tf.gradients(mse, [theta2])[0] # 使用tf自带函数进行导数计算

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch', epoch, "MSE =", mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()
print(best_theta)

Epoch 0 MSE = 11.122785
Epoch 100 MSE = 0.83446014
Epoch 200 MSE = 0.6245525
Epoch 300 MSE = 0.5986512
Epoch 400 MSE = 0.58227843
Epoch 500 MSE = 0.56985044
Epoch 600 MSE = 0.56027126
Epoch 700 MSE = 0.5528437
Epoch 800 MSE = 0.5470534
Epoch 900 MSE = 0.5425173
[[ 2.0685523 ]
 [ 0.91460514]
 [ 0.15763874]
 [-0.3831629 ]
 [ 0.38409027]
 [ 0.00853981]
 [-0.04442774]
 [-0.5365979 ]
 [-0.51506793]]
CPU times: user 469 ms, sys: 52.6 ms, total: 521 ms
Wall time: 432 ms


### Using an Optimizer

In [34]:
%%time
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch', epoch, "MSE =", mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()
print(best_theta)

Epoch 0 MSE = 7.4996443
Epoch 100 MSE = 0.8572574
Epoch 200 MSE = 0.700979
Epoch 300 MSE = 0.6636375
Epoch 400 MSE = 0.63595164
Epoch 500 MSE = 0.6141535
Epoch 600 MSE = 0.596878
Epoch 700 MSE = 0.5831238
Epoch 800 MSE = 0.57212394
Epoch 900 MSE = 0.5632915
[[ 2.0685525 ]
 [ 0.9814844 ]
 [ 0.17175326]
 [-0.5065767 ]
 [ 0.48484865]
 [ 0.0128072 ]
 [-0.04704794]
 [-0.370915  ]
 [-0.35702637]]
CPU times: user 466 ms, sys: 47.8 ms, total: 514 ms
Wall time: 430 ms


In [9]:
%%time
n_epochs = 1000
learning_rate = 0.01
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing_target, dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradient = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

# momentum 动量？
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9) # 替换了这里，优化
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch', epoch, "MSE =", mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()
print(best_theta)

Epoch 0 MSE = 9.102093
Epoch 100 MSE = 0.5379996
Epoch 200 MSE = 0.5254084
Epoch 300 MSE = 0.5244528
Epoch 400 MSE = 0.5243385
Epoch 500 MSE = 0.5243229
Epoch 600 MSE = 0.524321
Epoch 700 MSE = 0.52432096
Epoch 800 MSE = 0.52432126
Epoch 900 MSE = 0.52432084
[[ 2.0685577 ]
 [ 0.82963055]
 [ 0.11875372]
 [-0.26554817]
 [ 0.30571374]
 [-0.00450239]
 [-0.03932669]
 [-0.89986   ]
 [-0.8705167 ]]
CPU times: user 485 ms, sys: 53 ms, total: 538 ms
Wall time: 440 ms
